In [11]:
import cobra
from cobra import Metabolite, Gene, Reaction
from cobra.flux_analysis import single_gene_deletion
import pandas as pd
from medusa.test import load_universal_modelseed
import medusa
from pickle import load
from medusa.flux_analysis import deletion

In [12]:
with open("../results/ensembles/psy_ensemble_100_for_analysis.pickle", 'rb') as infile:
    ensemble = load(infile)
#universal = load_universal_modelseed()

In [13]:
#set media conditions for each ensemble

#complete media has all of the exchange reactions open


        
#for the other medias, use teh base biolog salts (minus heme) and add the media types needed
from medusa.test import load_biolog_plata
biolog_base_composition, biolog_base_dict, biolog_thresholded = load_biolog_plata()
biolog_base_composition = biolog_base_composition.drop(18)
biolog_base_composition = biolog_base_composition.drop(19)

complete = {}
for reaction in ensemble.base_model.reactions:
    if "EX_" in reaction.id:
        complete[reaction.id] = 10
for x in biolog_base_composition.ID:
    complete['EX_' + x] = 1000

gluc_mm = {}

for x in biolog_base_composition.ID:
    gluc_mm['EX_' + x] = 1000 
gluc_mm['EX_cpd00027_e'] = 10

glut_mm = {}

for x in biolog_base_composition.ID:
    glut_mm['EX_' + x] = 1000
glut_mm['EX_cpd00053_e'] = 10

#load in the anderson metabolites
anderson_df = pd.read_csv("../data/Anderson_2014/metabolite_ids_Anderson2014.csv")
anderson_media_list = []
met_ids = [met for met in anderson_df.met_id]
for met in met_ids:
    met="EX_" + str(met) + "_e"
    anderson_media_list.append(met)
    
    
anderson = {}
for x in biolog_base_composition.ID:
    anderson['EX_' + x] = 1000 
for y in anderson_media_list:
    if y in ensemble.base_model.reactions:
        anderson[y] = 10
        #print (y + " is IN the model")
    else:
        pass
        #print (y+ " is not in the base model") 

suc_mm = {}
for x in biolog_base_composition.ID:
    suc_mm['EX_' + x] = 1000
suc_mm['EX_cpd00076_e'] = 10

leu_mm = {}
for x in biolog_base_composition.ID:
    leu_mm['EX_' + x] = 1000
leu_mm['EX_cpd00107_e'] = 10

glucose_mm = {}
for x in biolog_base_composition.ID:
    glucose_mm['EX_' + x] = 1000
glucose_mm['EX_cpd00027_e'] = 10

asp_mm = {}
for x in biolog_base_composition.ID:
    asp_mm['EX_' + x] = 1000
asp_mm['EX_cpd00041_e'] = 10

cit_mm = {}
for x in biolog_base_composition.ID:
    cit_mm['EX_' + x] = 1000
cit_mm['EX_cpd00137_e'] = 10


gaba_mm = {}
for x in biolog_base_composition.ID:
    gaba_mm['EX_' + x] = 1000
gaba_mm['EX_cpd00281_e'] = 10

fru_mm = {}
for x in biolog_base_composition.ID:
    fru_mm['EX_' + x] = 1000
fru_mm['EX_cpd00082_e'] = 10

media_list = {'complete':complete, 'glucose_mm':glucose_mm, 'glut_mm':glut_mm, 'anderson':anderson, 'suc_mm':suc_mm, 'asp_mm':asp_mm, 'cit_mm':cit_mm, 'gaba_mm':gaba_mm, 'fru_mm':fru_mm}

In [4]:
#set media conditions for each ensemble
#this is for a gradient of leucine minimal and glucose minimal mixing
from medusa.test import load_biolog_plata
biolog_base_composition, biolog_base_dict, biolog_thresholded = load_biolog_plata()
biolog_base_composition = biolog_base_composition.drop(18)
biolog_base_composition = biolog_base_composition.drop(19)
l = {}
for x in biolog_base_composition.ID:
    l['EX_' + x] = 1000 
l['EX_cpd00107_e'] = 10

g = {}
for x in biolog_base_composition.ID:
    g['EX_' + x] = 1000 
g['EX_cpd00027_e'] = 10

l1 = {}
for x in biolog_base_composition.ID:
    l1['EX_' + x] = 1000 
l1['EX_cpd00027_e'] = 10
l1['EX_cpd00107_e'] = 0.1

l2 = {}
for x in biolog_base_composition.ID:
    l2['EX_' + x] = 1000 
l2['EX_cpd00027_e'] = 10
l2['EX_cpd00107_e'] = 1

l3 = {}
for x in biolog_base_composition.ID:
    l3['EX_' + x] = 1000 
l3['EX_cpd00027_e'] = 10
l3['EX_cpd00107_e'] = 10

l4 = {}
for x in biolog_base_composition.ID:
    l4['EX_' + x] = 1000 
l4['EX_cpd00027_e'] = 10
l4['EX_cpd00107_e'] = 100

l5 = {}
for x in biolog_base_composition.ID:
    l5['EX_' + x] = 1000 
l5['EX_cpd00027_e'] = 10
l5['EX_cpd00107_e'] = 1000

g1 = {}
for x in biolog_base_composition.ID:
    g1['EX_' + x] = 1000 
g1['EX_cpd00027_e'] = 0.1
g1['EX_cpd00107_e'] = 10

g2 = {}
for x in biolog_base_composition.ID:
    g2['EX_' + x] = 1000 
g2['EX_cpd00027_e'] = 1
g2['EX_cpd00107_e'] = 10

g3 = {}
for x in biolog_base_composition.ID:
    g3['EX_' + x] = 1000 
g3['EX_cpd00027_e'] = 100
g3['EX_cpd00107_e'] = 10

g4 = {}
for x in biolog_base_composition.ID:
    g4['EX_' + x] = 1000 
g4['EX_cpd00027_e'] = 1000
g4['EX_cpd00107_e'] = 10

In [3]:
#these media conditions are for restriction medias with leucine and glucose restriction

from medusa.test import load_biolog_plata
biolog_base_composition, biolog_base_dict, biolog_thresholded = load_biolog_plata()
biolog_base_composition = biolog_base_composition.drop(18)
biolog_base_composition = biolog_base_composition.drop(19)
l = {}
for x in biolog_base_composition.ID:
    l['EX_' + x] = 1000 
l['EX_cpd00107_e'] = 10

g = {}
for x in biolog_base_composition.ID:
    g['EX_' + x] = 1000 
g['EX_cpd00027_e'] = 10

l1 = {}
for x in biolog_base_composition.ID:
    l1['EX_' + x] = 1000 
l1['EX_cpd00107_e'] = 9.9
l1['EX_cpd00027_e'] = 0.1

lg = {}
for x in biolog_base_composition.ID:
    lg['EX_' + x] = 1000 
lg['EX_cpd00107_e'] = 5
lg['EX_cpd00027_e'] = 5

g1 = {}
for x in biolog_base_composition.ID:
    g1['EX_' + x] = 1000 
g1['EX_cpd00107_e'] = 0.1
g1['EX_cpd00027_e'] = 9.9

media_dict = {'l':l, 'l1':l1, 'lg':lg, 'g1':g1, 'g' :g}

In [13]:
#this media condition set is for the gradient of leucine and glucose
from medusa.test import load_biolog_plata
biolog_base_composition, biolog_base_dict, biolog_thresholded = load_biolog_plata()
biolog_base_composition = biolog_base_composition.drop(18)
biolog_base_composition = biolog_base_composition.drop(19)
l1000 = {}
for x in biolog_base_composition.ID:
    l1000['EX_' + x] = 1000 
l1000['EX_cpd00107_e'] = 1000

l500 = {}
for x in biolog_base_composition.ID:
    l500['EX_' + x] = 1000 
l500['EX_cpd00107_e'] = 500
l500['EX_cpd00027_e'] = 0.1

l100 = {}
for x in biolog_base_composition.ID:
    l100['EX_' + x] = 1000 
l100['EX_cpd00107_e'] = 100
l100['EX_cpd00027_e'] = 1

l10 = {}
for x in biolog_base_composition.ID:
    l10['EX_' + x] = 1000 
l10['EX_cpd00107_e'] = 10
l10['EX_cpd00027_e'] = 10

l1_gradient = {}
for x in biolog_base_composition.ID:
    l1_gradient['EX_' + x] = 1000 
l1_gradient['EX_cpd00107_e'] = 1
l1_gradient['EX_cpd00027_e'] = 100

l01 = {}
for x in biolog_base_composition.ID:
    l01['EX_' + x] = 1000 
l01['EX_cpd00107_e'] = 0.1
l01['EX_cpd00027_e'] = 500

l0 = {}
for x in biolog_base_composition.ID:
    l0['EX_' + x] = 1000 
l0['EX_cpd00027_e'] = 1000

media_dict = {'l0':l0, 'l01':l01, 'l0_gradient':l1_gradient, 'l10':l10, 'l100' :l100, 'l500':l500, 'l1000': l1000}

In [6]:
model_list = []
for model in ensemble.members:
    model_list.append(model.id)

In [9]:
for x,y in media_list.items():
    ensemble.base_model.medium = y
    filevar = x
    file_path = '../results/gene_essentiality/ensemble_essentialiity/'
    file_suf_xl = '_media_ensemble_deletions.xlsx'
    file_name_ensemble_deletions = file_path + filevar + file_suf_xl
    deletion_results = {}
    with ensemble.base_model:
        for model in model_list:
            print('performing deletions for ' + model)
            ensemble.set_state(model)
            deletion_result = single_gene_deletion(ensemble.base_model)
            deletion_results[model] = deletion_result

    with pd.ExcelWriter(file_name_ensemble_deletions) as writer:  # doctest: +SKIP
        for key, value in deletion_results.items():
            #print (key)
            value.to_excel(writer, sheet_name=key)
    reps = {}
    for i in range(-1,99):
        gene_knockouts = pd.read_excel(file_name_ensemble_deletions ,sheet_name="PST_gapfilled_"+str(i+1))
        gene_knockouts.columns = [col for col in gene_knockouts.columns]
        reps[i] = gene_knockouts
    all_biolog = pd.concat([reps[-1], reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                            , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                            , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                            , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                            , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                            , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                            , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])


    file_suf_csv = '_media_ensemble_deletions.csv'
    file_name_concat =file_path + filevar + "concat" + file_suf_csv
    file_name_unique =file_path + filevar + "unique" + file_suf_csv


    all_biolog.to_csv(file_name_concat)
    df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
    df3.to_csv(file_name_unique)

    print ("I have completed: " + filevar)


EX_cpd11416_c does not seem to be an an exchange reaction. Applying bounds anyway.


performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [15]:
ensemble.base_model.medium = complete
filevar = 'complete'
file_path = '../results/gene_essentiality/ensemble_essentialiity/'
file_suf_xl = '_media_ensemble_deletions.xlsx'
file_name_ensemble_deletions = file_path + filevar + file_suf_xl
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter(file_name_ensemble_deletions) as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        #print (key)
        value.to_excel(writer, sheet_name=key)
reps = {}
for i in range(-1,99):
    gene_knockouts = pd.read_excel(file_name_ensemble_deletions ,sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[-1], reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])


file_suf_csv = '_media_ensemble_deletions.csv'
file_name_concat =file_path + filevar + "concat" + file_suf_csv
file_name_unique =file_path + filevar + "unique" + file_suf_csv


all_biolog.to_csv(file_name_concat)
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv(file_name_unique)

print ("I have completed: " + filevar)
#manually determine the essential genes using count if function, save as same name excel extention

EX_cpd11416_c does not seem to be an an exchange reaction. Applying bounds anyway.


performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [16]:
complete

{'EX_cpd16565_e': 10,
 'EX_cpd00001_e': 1000,
 'EX_cpd00007_e': 1000,
 'EX_cpd00009_e': 1000,
 'EX_cpd00011_e': 1000,
 'EX_cpd00013_e': 1000,
 'EX_cpd00030_e': 1000,
 'EX_cpd00034_e': 1000,
 'EX_cpd00048_e': 1000,
 'EX_cpd00058_e': 1000,
 'EX_cpd00063_e': 1000,
 'EX_cpd00067_e': 1000,
 'EX_cpd00099_e': 1000,
 'EX_cpd00149_e': 1000,
 'EX_cpd00205_e': 1000,
 'EX_cpd00254_e': 1000,
 'EX_cpd00971_e': 1000,
 'EX_cpd10515_e': 1000,
 'EX_cpd10516_e': 1000,
 'EX_cpd00224_e': 10,
 'EX_cpd00280_e': 10,
 'EX_cpd00666_e': 10,
 'EX_cpd00129_e': 10,
 'EX_cpd00249_e': 10,
 'EX_cpd00027_e': 10,
 'EX_cpd00023_e': 10,
 'EX_cpd00082_e': 10,
 'EX_cpd00100_e': 10,
 'EX_cpd00246_e': 10,
 'EX_cpd00020_e': 10,
 'EX_cpd01293_e': 10,
 'EX_cpd00138_e': 10,
 'EX_cpd00130_e': 10,
 'EX_cpd00386_e': 10,
 'EX_cpd00106_e': 10,
 'EX_cpd00036_e': 10,
 'EX_cpd00105_e': 10,
 'EX_cpd01307_e': 10,
 'EX_cpd00108_e': 10,
 'EX_cpd00121_e': 10,
 'EX_cpd00609_e': 10,
 'EX_cpd00154_e': 10,
 'EX_cpd00107_e': 10,
 'EX_cpd00222_e': 

In [18]:
for x,y in media_dict.items():
    filevar = str(x)
    import re
    file_path = '../results/gene_essentiality/ensemble_essentialiity/'
    file_suf_xl = '_media_ensemble_deletions.xlsx'
    file_name = file_path+ 'leu_glu_media_gradient/' + filevar + "unique" + file_suf_xl
    complete = pd.read_excel(file_name)
    essen = complete.query('counts_of==0')['ids']
    file_name = file_path + filevar + "essential_only" + file_suf_csv
    essen.to_csv(file_name)
    patric_path = file_path + "PATRIC_id_mapping.csv"
    patric = pd.read_csv(patric_path)
    patric_dict = patric.to_dict(orient = 'index')
    complete = pd.read_csv(file_name, names = ["ids"])
    complete_ids = [ids for ids in complete.ids]

    complete_gene_list=[]
    for x in complete_ids:
        x = str(x)
        if 'nan' in x:
            pass
        else:
            result = re.split("frozenset\\({'", x)
        for y in result:
            if "frozenset" in y:
                pass

            else:
                complete_gene_list.append(str(y[:-3]))
    pd.DataFrame(complete_gene_list)
    file_suf_r = "for_r.csv"
    file_name = file_name_unique =file_path + filevar + "unique" + file_suf_r
    unique_list = []

    #print (complete_gene_list)
    complete_list_mod = [] 
    for z in complete_gene_list:
        z = "fig|" + z
        complete_list_mod.append(z)
    #print (complete_list_mod)
    new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
    patric_genes_essential = []
    for key,value in new_pat.items():
        if key in complete_list_mod:
            #print (key, value)
            patric_genes_essential.append(value)
    complete_and_patric_gene_list = patric_genes_essential + complete_gene_list

    for z in complete_and_patric_gene_list:
        if z not in unique_list:
            if 'PSPTO_' in z:
                z = "pst:" + z
                unique_list.append(z)
            else: pass
        else:
            pass
    listo = pd.DataFrame(unique_list)
    listo.to_csv(file_name)
    print (filevar + " is ready for R.")

/Users/phil/cobrapython/lib/python3.7/site-packages/ipykernel_launcher.py:10 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


l0 is ready for R.
l01 is ready for R.
l0_gradient is ready for R.
l10 is ready for R.
l100 is ready for R.
l500 is ready for R.
l1000 is ready for R.


In [13]:
filevar = 'suc_mm_100_for_analysis'
file_name = file_path + filevar + "_orthologies.csv"
kegg_list_super = pd.read_excel('../results/gene_essentiality/ensemble_essentialiity/kegg_list_super.xlsx', names = ['Gene', 'Pathway'])
kegg_list_super_dict = pd.Series(kegg_list_super.Pathway.values,index=kegg_list_super.Gene).to_dict()
complete = pd.read_csv((file_name), names = ['Gene', 'Pathway'])
complete_p_count = complete.groupby(['Pathway']).count()
file_name = file_path + filevar + "_orthologies_counted.csv"
complete_p_count.to_csv(file_name)
complete_p_count.reset_index(level=0, inplace=True)
complete_p_count['supergroup'] = complete_p_count['Pathway'].map(kegg_list_super_dict)
file_name = file_path + filevar + "_supergrouped_orthologies.csv"
complete_p_count.to_csv(file_name)

In [116]:
file_name = file_path + filevar + "unique" + file_suf_xl
complete = pd.read_excel(file_name)
essen = complete.query('counts_of==0')['ids']
file_name = file_path + filevar + "essential_only" + file_suf_csv
essen.to_csv(file_name)
#for row in complete.itertuples():
    #print (complete.counts_of)
    #x = str(complete.counts_of)
    #print (x)
    #if "0" is x:
     #   print ('yes')
        


/Users/phil/cobrapython/lib/python3.7/site-packages/ipykernel_launcher.py:5 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [9]:
filevar = 'suc_mm_100_for_analysis'
file_path = '../results/gene_essentiality/ensemble_essentialiity/'
file_suf_xl = '_media_ensemble_deletions.xlsx'
patric_path = file_path + "PATRIC_id_mapping.csv"
patric = pd.read_csv(patric_path)
patric_dict = patric.to_dict(orient = 'index')
complete = pd.read_csv(file_name, names = ["ids"])
complete_ids = [ids for ids in complete.ids]
for y in complete_ids:
    if "nan":
        pass
    else:
        print (y)
complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass

        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)
file_suf_r = "for_r.csv"
file_name = file_name_unique =file_path + filevar + "unique" + file_suf_r
unique_list = []

#print (complete_gene_list)
complete_list_mod = [] 
for z in complete_gene_list:
    z = "fig|" + z
    complete_list_mod.append(z)
#print (complete_list_mod)
new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
patric_genes_essential = []
for key,value in new_pat.items():
    if key in complete_list_mod:
        print (key, value)
        patric_genes_essential.append(value)
complete_and_patric_gene_list = patric_genes_essential + complete_gene_list

for z in complete_and_patric_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv(file_name)

#after r, use the next cell

FileNotFoundError: [Errno 2] File b'../../../Desktop/all_genesuniquefor_r.csv' does not exist: b'../../../Desktop/all_genesuniquefor_r.csv'

In [115]:
#print (complete_gene_list)
complete_list_mod = [] 
for z in complete_gene_list:
    z = "fig|" + z
    complete_list_mod.append(z)
#print (complete_list_mod)
new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
patric_genes_essential = []
for key,value in new_pat.items():
    if key in complete_list_mod:
        print (key, value)
        patric_genes_essential.append(value)
complete_and_patric_gene_list = patric_genes_essential + complete_gene_list


fig|223283.9.peg.1 PSPTO_0001
fig|223283.9.peg.1290 PSPTO_1268
fig|223283.9.peg.1494 PSPTO_1473
fig|223283.9.peg.1497 PSPTO_1476
fig|223283.9.peg.1522 PSPTO_1501
fig|223283.9.peg.1560 PSPTO_1534
fig|223283.9.peg.1575 PSPTO_1549
fig|223283.9.peg.1577 PSPTO_1551
fig|223283.9.peg.1663 PSPTO_1639
fig|223283.9.peg.1773 PSPTO_1745
fig|223283.9.peg.184 PSPTO_0178
fig|223283.9.peg.1841 PSPTO_1812
fig|223283.9.peg.1842 PSPTO_1813
fig|223283.9.peg.1871 PSPTO_1842
fig|223283.9.peg.190 PSPTO_0184
fig|223283.9.peg.191 PSPTO_0185
fig|223283.9.peg.2 PSPTO_0002
fig|223283.9.peg.2197 PSPTO_2166
fig|223283.9.peg.2413 PSPTO_2378
fig|223283.9.peg.2415 PSPTO_2380
fig|223283.9.peg.2416 PSPTO_2381
fig|223283.9.peg.2417 PSPTO_2382
fig|223283.9.peg.2418 PSPTO_2383
fig|223283.9.peg.3424 PSPTO_3345
fig|223283.9.peg.3599 PSPTO_3514
fig|223283.9.peg.3735 PSPTO_3646
fig|223283.9.peg.3748 PSPTO_3658
fig|223283.9.peg.3813 PSPTO_3721
fig|223283.9.peg.3834 PSPTO_3742
fig|223283.9.peg.3923 PSPTO_3826
fig|223283.9.peg.39

In [33]:
ensemble.base_model.medium = glut_mm
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter('../results/gene_essentiality/ensemble_essentialiity/glut_media_ensemble_deletions.xlsx') as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        print (key)
        value.to_excel(writer, sheet_name=key)

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [38]:
reps = {}
for i in range(0,99):
    gene_knockouts = pd.read_excel('../results/gene_essentiality/ensemble_essentialiity/glut_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])

all_biolog.to_csv('../results/gene_essentiality/ensemble_essentialiity/glut_essentiality.csv')
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv('../results/gene_essentiality/ensemble_essentialiity/unique_glut.csv')

In [41]:
ensemble.base_model.medium = gluc_mm
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter('../results/gene_essentiality/ensemble_essentialiity/gluc_media_ensemble_deletions.xlsx') as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        value.to_excel(writer, sheet_name=key)


performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [43]:
reps = {}
for i in range(0,99):
    gene_knockouts = pd.read_excel('../results/gene_essentiality/ensemble_essentialiity/gluc_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])

all_biolog.to_csv('../results/gene_essentiality/ensemble_essentialiity/gluc_essentiality.csv')
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv('../results/gene_essentiality/ensemble_essentialiity/unique_gluc.csv')

In [42]:
ensemble.base_model.medium = anderson
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter('../results/gene_essentiality/ensemble_essentialiity/anderson_media_ensemble_deletions.xlsx') as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        print (key)
        value.to_excel(writer, sheet_name=key)

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [44]:
reps = {}
for i in range(0,99):
    gene_knockouts = pd.read_excel('../results/gene_essentiality/ensemble_essentialiity/anderson_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])

all_biolog.to_csv('../results/gene_essentiality/ensemble_essentialiity/anderson_essentiality.csv')
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv('../results/gene_essentiality/ensemble_essentialiity/unique_anderson.csv')

In [6]:
model = ensemble.extract_member("PST_gapfilled_24")

In [7]:
cobra.io.write_sbml_model(model, "pst_24.xml")

Error encountered trying to <set model history>.
LibSBML error code -5: The object passed as an argument to the method is not of a type that is valid for the operation or kind of object involved. For example, handing an invalidly-constructed ASTNode to a method expecting an ASTNode will result in this error.


In [9]:
model2 = cobra.io.read_sbml_model("pst_24.xml")

In [46]:
import re
complete = pd.read_excel("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/ensemble_1_essen_compile.xlsx")
complete_ids = [ids for ids in complete.complete]
for y in complete_ids:
    if "nan":
        pass
    else:
        print (y)
complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass
        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)

#change this for r, so that the keggrest can use it
unique_list = []
for z in complete_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/unique_complete_essential_all_for_r.csv")
complete_ids = [ids for ids in complete.anderson]
for y in complete_ids:
    if "nan":
        pass
    else:
        print (y)
complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass
        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)

#change this for r, so that the keggrest can use it
unique_list = []
for z in complete_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/unique_anderson_essential_all_for_r.csv")
complete_ids = [ids for ids in complete.glut]
for y in complete_ids:
    if "nan":
        pass
    else:
        print (y)
complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass
        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)

#change this for r, so that the keggrest can use it
unique_list = []
for z in complete_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/unique_glut_essential_all_for_r.csv")
complete_ids = [ids for ids in complete.gluc]
for y in complete_ids:
    if "nan":
        pass
    else:
        print (y)
complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass
        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)

#change this for r, so that the keggrest can use it
unique_list = []
for z in complete_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/unique_gluc_essential_all_for_r.csv")

In [48]:
#map genes to patric identifiers

patric = pd.read_csv("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/PATRIC_id_mapping.csv")

In [56]:
pat_dict = {}
for x in patric.Source:
    pat_dict[x] = patric.Target

In [57]:
pat_dict

{'fig|223283.9.peg.1': 0      PSPTO_0001
 1      PSPTO_1005
 2      PSPTO_1041
 3      PSPTO_1051
 4      PSPTO_1053
 5      PSPTO_0103
 6      PSPTO_1077
 7      PSPTO_1079
 8      PSPTO_1080
 9      PSPTO_1081
 10     PSPTO_1104
 11     PSPTO_1105
 12     PSPTO_1108
 13     PSPTO_1112
 14     PSPTO_1128
 15     PSPTO_1129
 16     PSPTO_1131
 17     PSPTO_1136
 18     PSPTO_0112
 19     PSPTO_1169
 20     PSPTO_1178
 21     PSPTO_1182
 22     PSPTO_1196
 23     PSPTO_0118
 24     PSPTO_1215
 25     PSPTO_1232
 26     PSPTO_1243
 27     PSPTO_1268
 28     PSPTO_1271
 29     PSPTO_1276
           ...    
 688    PSPTO_0800
 689    PSPTO_0802
 690    PSPTO_0805
 691    PSPTO_0806
 692    PSPTO_0809
 693    PSPTO_0817
 694    PSPTO_0080
 695    PSPTO_0836
 696    PSPTO_0083
 697    PSPTO_0085
 698    PSPTO_0885
 699    PSPTO_0886
 700    PSPTO_0089
 701    PSPTO_0923
 702    PSPTO_0090
 703    PSPTO_0948
 704    PSPTO_0092
 705    PSPTO_0954
 706    PSPTO_0955
 707    PSPTO_0956
 708    P

In [25]:
filevars = ['biolog_base', 'complete' , 'ile' , 'hmm' , 'glut' , 'gluc' , 'val' , 'anderson']
filesuff = '_100_e3_supergrouped_orthologies.csv'
filepre = '../results/gene_essentiality/ensemble_essentialiity/'
for x in filevars:  
    filevar = x
    filename = filepre + x + filesuff
    table = pd.read_csv(filename)
    

 

In [8]:
ensemble.base_model.medium = glucose_mm
filevar = 'glucose_mm_100_for_analysis'
file_path = '../results/gene_essentiality/ensemble_essentialiity/'
file_suf_xl = '_media_ensemble_deletions.xlsx'
file_name_ensemble_deletions = file_path + filevar + file_suf_xl
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter(file_name_ensemble_deletions) as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        #print (key)
        value.to_excel(writer, sheet_name=key)
reps = {}
for i in range(0,99):
    gene_knockouts = pd.read_excel(file_name_ensemble_deletions ,sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])


file_suf_csv = '_media_ensemble_deletions.csv'
file_name_concat =file_path + filevar + "concat" + file_suf_csv
file_name_unique =file_path + filevar + "unique" + file_suf_csv


all_biolog.to_csv(file_name_concat)
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv(file_name_unique)

print ("I have completed: " + filevar)
#manually determine the essential genes using count if function, save as same name excel extention

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [15]:
filevar = 'glucose_mm_100_for_analysis'
import re
file_name = file_path + filevar + "unique" + file_suf_xl
complete = pd.read_excel(file_name)
essen = complete.query('counts_of==0')['ids']
file_name = file_path + filevar + "essential_only" + file_suf_csv
essen.to_csv(file_name)
patric_path = file_path + "PATRIC_id_mapping.csv"
patric = pd.read_csv(patric_path)
patric_dict = patric.to_dict(orient = 'index')
complete = pd.read_csv(file_name, names = ["ids"])
complete_ids = [ids for ids in complete.ids]

complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass

        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)
file_suf_r = "for_r.csv"
file_name = file_name_unique =file_path + filevar + "unique" + file_suf_r
unique_list = []

#print (complete_gene_list)
complete_list_mod = [] 
for z in complete_gene_list:
    z = "fig|" + z
    complete_list_mod.append(z)
#print (complete_list_mod)
new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
patric_genes_essential = []
for key,value in new_pat.items():
    if key in complete_list_mod:
        #print (key, value)
        patric_genes_essential.append(value)
complete_and_patric_gene_list = patric_genes_essential + complete_gene_list

for z in complete_and_patric_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv(file_name)
print (filevar + " is ready for R.")

glucose_mm_100_for_analysis is ready for R.


/Users/phil/cobrapython/lib/python3.7/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [9]:
ensemble.base_model.medium = asp_mm
filevar = 'asp_mm_100_for_analysis'
file_path = '../results/gene_essentiality/ensemble_essentialiity/'
file_suf_xl = '_media_ensemble_deletions.xlsx'
file_name_ensemble_deletions = file_path + filevar + file_suf_xl
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter(file_name_ensemble_deletions) as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        #print (key)
        value.to_excel(writer, sheet_name=key)
reps = {}
for i in range(0,99):
    gene_knockouts = pd.read_excel(file_name_ensemble_deletions ,sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])


file_suf_csv = '_media_ensemble_deletions.csv'
file_name_concat =file_path + filevar + "concat" + file_suf_csv
file_name_unique =file_path + filevar + "unique" + file_suf_csv


all_biolog.to_csv(file_name_concat)
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv(file_name_unique)

print ("I have completed: " + filevar)
#manually determine the essential genes using count if function, save as same name excel extention

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [14]:
filevar = 'asp_mm_100_for_analysis'
import re
file_name = file_path + filevar + "unique" + file_suf_xl
complete = pd.read_excel(file_name)
essen = complete.query('counts_of==0')['ids']
file_name = file_path + filevar + "essential_only" + file_suf_csv
essen.to_csv(file_name)
patric_path = file_path + "PATRIC_id_mapping.csv"
patric = pd.read_csv(patric_path)
patric_dict = patric.to_dict(orient = 'index')
complete = pd.read_csv(file_name, names = ["ids"])
complete_ids = [ids for ids in complete.ids]

complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass

        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)
file_suf_r = "for_r.csv"
file_name = file_name_unique =file_path + filevar + "unique" + file_suf_r
unique_list = []

#print (complete_gene_list)
complete_list_mod = [] 
for z in complete_gene_list:
    z = "fig|" + z
    complete_list_mod.append(z)
#print (complete_list_mod)
new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
patric_genes_essential = []
for key,value in new_pat.items():
    if key in complete_list_mod:
        #print (key, value)
        patric_genes_essential.append(value)
complete_and_patric_gene_list = patric_genes_essential + complete_gene_list

for z in complete_and_patric_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv(file_name)
print (filevar + " is ready for R.")

asp_mm_100_for_analysis is ready for R.


/Users/phil/cobrapython/lib/python3.7/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [10]:
ensemble.base_model.medium = cit_mm
filevar = 'cit_mm_100_for_analysis'
file_path = '../results/gene_essentiality/ensemble_essentialiity/'
file_suf_xl = '_media_ensemble_deletions.xlsx'
file_name_ensemble_deletions = file_path + filevar + file_suf_xl
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter(file_name_ensemble_deletions) as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        #print (key)
        value.to_excel(writer, sheet_name=key)
reps = {}
for i in range(0,99):
    gene_knockouts = pd.read_excel(file_name_ensemble_deletions ,sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])


file_suf_csv = '_media_ensemble_deletions.csv'
file_name_concat =file_path + filevar + "concat" + file_suf_csv
file_name_unique =file_path + filevar + "unique" + file_suf_csv


all_biolog.to_csv(file_name_concat)
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv(file_name_unique)

print ("I have completed: " + filevar)
#manually determine the essential genes using count if function, save as same name excel extention

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [13]:
filevar = 'cit_mm_100_for_analysis'
import re
file_name = file_path + filevar + "unique" + file_suf_xl
complete = pd.read_excel(file_name)
essen = complete.query('counts_of==0')['ids']
file_name = file_path + filevar + "essential_only" + file_suf_csv
essen.to_csv(file_name)
patric_path = file_path + "PATRIC_id_mapping.csv"
patric = pd.read_csv(patric_path)
patric_dict = patric.to_dict(orient = 'index')
complete = pd.read_csv(file_name, names = ["ids"])
complete_ids = [ids for ids in complete.ids]

complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass

        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)
file_suf_r = "for_r.csv"
file_name = file_name_unique =file_path + filevar + "unique" + file_suf_r
unique_list = []

#print (complete_gene_list)
complete_list_mod = [] 
for z in complete_gene_list:
    z = "fig|" + z
    complete_list_mod.append(z)
#print (complete_list_mod)
new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
patric_genes_essential = []
for key,value in new_pat.items():
    if key in complete_list_mod:
        #print (key, value)
        patric_genes_essential.append(value)
complete_and_patric_gene_list = patric_genes_essential + complete_gene_list

for z in complete_and_patric_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv(file_name)
print (filevar + " is ready for R.")

cit_mm_100_for_analysis is ready for R.


/Users/phil/cobrapython/lib/python3.7/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [11]:
ensemble.base_model.medium = gaba_mm
filevar = 'gaba_mm_100_for_analysis'
file_path = '../results/gene_essentiality/ensemble_essentialiity/'
file_suf_xl = '_media_ensemble_deletions.xlsx'
file_name_ensemble_deletions = file_path + filevar + file_suf_xl
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter(file_name_ensemble_deletions) as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        #print (key)
        value.to_excel(writer, sheet_name=key)
reps = {}
for i in range(0,99):
    gene_knockouts = pd.read_excel(file_name_ensemble_deletions ,sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])


file_suf_csv = '_media_ensemble_deletions.csv'
file_name_concat =file_path + filevar + "concat" + file_suf_csv
file_name_unique =file_path + filevar + "unique" + file_suf_csv


all_biolog.to_csv(file_name_concat)
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv(file_name_unique)

print ("I have completed: " + filevar)
#manually determine the essential genes using count if function, save as same name excel extention

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [12]:
filevar = 'gaba_mm_100_for_analysis'
import re
file_name = file_path + filevar + "unique" + file_suf_xl
complete = pd.read_excel(file_name)
essen = complete.query('counts_of==0')['ids']
file_name = file_path + filevar + "essential_only" + file_suf_csv
essen.to_csv(file_name)
patric_path = file_path + "PATRIC_id_mapping.csv"
patric = pd.read_csv(patric_path)
patric_dict = patric.to_dict(orient = 'index')
complete = pd.read_csv(file_name, names = ["ids"])
complete_ids = [ids for ids in complete.ids]

complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass

        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)
file_suf_r = "for_r.csv"
file_name = file_name_unique =file_path + filevar + "unique" + file_suf_r
unique_list = []

#print (complete_gene_list)
complete_list_mod = [] 
for z in complete_gene_list:
    z = "fig|" + z
    complete_list_mod.append(z)
#print (complete_list_mod)
new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
patric_genes_essential = []
for key,value in new_pat.items():
    if key in complete_list_mod:
        #print (key, value)
        patric_genes_essential.append(value)
complete_and_patric_gene_list = patric_genes_essential + complete_gene_list

for z in complete_and_patric_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv(file_name)
print (filevar + " is ready for R.")

gaba_mm_100_for_analysis is ready for R.


/Users/phil/cobrapython/lib/python3.7/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [10]:
import re

filevar = 'suc_mm_100_for_analysis'
file_path = '../results/gene_essentiality/ensemble_essentialiity/'
file_suf_xl = '_media_ensemble_deletions.xlsx'
patric_path = file_path + "PATRIC_id_mapping.csv"
patric = pd.read_csv(patric_path)



file_path = "../../../Desktop/"
filevar = "all_genes"
file_suf = '.xlsx'
complete_gene_list=[]
file_name = file_path +filevar + file_suf


complete = pd.read_excel(file_name)
complete_ids = [ids for ids in complete.ids]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass

        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)
file_suf_r = "for_r.csv"
file_name = file_name_unique =file_path + filevar + "unique" + file_suf_r
unique_list = []

#print (complete_gene_list)
complete_list_mod = [] 
for z in complete_gene_list:
    z = "fig|" + z
    complete_list_mod.append(z)
#print (complete_list_mod)
new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
patric_genes_essential = []
for key,value in new_pat.items():
    if key in complete_list_mod:
        #print (key, value)
        patric_genes_essential.append(value)
complete_and_patric_gene_list = patric_genes_essential + complete_gene_list

for z in complete_and_patric_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv(file_name)
print (filevar + " is ready for R.")

all_genes is ready for R.


In [3]:
import pandas as pd
file_path = '../../../Desktop/'
filevar = 'all_genes'
file_name = file_path + filevar + "_orthologies.csv"
kegg_list_super = pd.read_excel('../results/gene_essentiality/ensemble_essentialiity/kegg_list_super.xlsx', names = ['Gene', 'Pathway'])
kegg_list_super_dict = pd.Series(kegg_list_super.Pathway.values,index=kegg_list_super.Gene).to_dict()
complete = pd.read_csv((file_name), names = ['Gene', 'Pathway'])
complete_p_count = complete.groupby(['Pathway']).count()
file_name = file_path + filevar + "_orthologies_counted.csv"
complete_p_count.to_csv(file_name)
complete_p_count.reset_index(level=0, inplace=True)
complete_p_count['supergroup'] = complete_p_count['Pathway'].map(kegg_list_super_dict)
file_name = file_path + filevar + "_supergrouped_orthologies.csv"
complete_p_count.to_csv(file_name)

In [15]:
ensemble.base_model.medium = l
filevar = 'l_100_for_analysis'
file_path = '../results/leu_gluc'
file_suf_xl = '_media_ensemble_deletions.xlsx'
file_name_ensemble_deletions = file_path + filevar + file_suf_xl
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter(file_name_ensemble_deletions) as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        #print (key)
        value.to_excel(writer, sheet_name=key)
reps = {}
for i in range(-1,99):
    gene_knockouts = pd.read_excel(file_name_ensemble_deletions ,sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[-1], reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])


file_suf_csv = '_media_ensemble_deletions.csv'
file_name_concat =file_path + filevar + "concat" + file_suf_csv
file_name_unique =file_path + filevar + "unique" + file_suf_csv


all_biolog.to_csv(file_name_concat)
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv(file_name_unique)

print ("I have completed: " + filevar)
#manually determine the essential genes using count if function, save as same name excel extention

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [25]:
#for genereating the list of ids for orthologies that are discordent between mixed and SCS medias with lecuine and glucose
import re
filevar = 'mixed_gl'
df = pd.read_excel('../results/gene_essentiality/leu_gluc/g_l_media_essentiality_counts_all.xlsx', sheetname='mixed')
complete_ids = df.values.tolist()
complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
        print (result)
    for y in result:
        if "frozenset" in y:
            pass
        else:
            complete_gene_list.append(str(y[:-5]))
print (complete_gene_list)
pd.DataFrame(complete_gene_list)
file_suf_r = "for_r.csv"
file_name = file_name_unique =file_path + filevar + "unique" + file_suf_r
unique_list = []

#print (complete_gene_list)
complete_list_mod = [] 
for z in complete_gene_list:
    z = "fig|" + z
    complete_list_mod.append(z)
#print (complete_list_mod)
new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
patric_genes_essential = []
for key,value in new_pat.items():
    if key in complete_list_mod:
        #print (key, value)
        patric_genes_essential.append(value)
complete_and_patric_gene_list = patric_genes_essential + complete_gene_list

for z in complete_and_patric_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv(file_name)
print (filevar + " is ready for R.")

['["', 'PSPTO_5150\'})"]']
['["', 'PSPTO_4560\'})"]']
['["', 'PSPTO_1298\'})"]']
['["', 'PSPTO_3009\'})"]']
['["', 'PSPTO_5261\'})"]']
['["', 'PSPTO_0073\'})"]']
['["', 'PSPTO_2719\'})"]']
['["', 'PSPTO_5494\'})"]']
['["', 'PSPTO_2957\'})"]']
['["', 'PSPTO_0782\'})"]']
['["', 'PSPTO_1136\'})"]']
['["', 'PSPTO_2730\'})"]']
['["', 'PSPTO_4682\'})"]']
['["', 'PSPTO_0518\'})"]']
['["', 'PSPTO_1275\'})"]']
['["', 'PSPTO_1441\'})"]']
['["', 'PSPTO_1508\'})"]']
['["', 'PSPTO_2453\'})"]']
['["', 'PSPTO_3499\'})"]']
['["', 'PSPTO_1172\'})"]']
['["', 'PSPTO_2028\'})"]']
['["', 'PSPTO_2981\'})"]']
['["', 'PSPTO_4201\'})"]']
['["', 'PSPTO_5047\'})"]']
['["', 'PSPTO_1688\'})"]']
['["', 'PSPTO_2043\'})"]']
['["', 'PSPTO_2200\'})"]']
['["', 'PSPTO_1183\'})"]']
['["', 'PSPTO_0730\'})"]']
['["', 'PSPTO_1822\'})"]']
['["', 'PSPTO_1300\'})"]']
['["', 'PSPTO_4866\'})"]']
['["', 'PSPTO_0169\'})"]']
['["', 'PSPTO_1053\'})"]']
['["', 'PSPTO_1553\'})"]']
['["', 'PSPTO_1713\'})"]']
['["', 'PSPTO_1720\'})"]']
[

In [33]:
file_path = '../../../Desktop/'
filevar = 'gl_mixed'
file_name = file_path + filevar + ".csv"
kegg_list_super = pd.read_excel('../results/gene_essentiality/ensemble_essentialiity/kegg_list_super.xlsx', names = ['Gene', 'Pathway'])
kegg_list_super_dict = pd.Series(kegg_list_super.Pathway.values,index=kegg_list_super.Gene).to_dict()
complete = pd.read_csv((file_name), names = ['Gene', 'Pathway'])
complete_p_count = complete.groupby(['Pathway']).count()
file_name = file_path + filevar + "_orthologies_counted.csv"
complete_p_count.to_csv(file_name)
complete_p_count.reset_index(level=0, inplace=True)
complete_p_count['supergroup'] = complete_p_count['Pathway'].map(kegg_list_super_dict)
file_name = file_path + filevar + "_supergrouped_orthologies.csv"
complete_p_count.to_csv(file_name)